Dependencies

In [ ]:
# !pip install faiss-cpu langchain pypdf sentence-transformers transformers


You should consider upgrading via the '/Volumes/nvme1TB_SSD/IdeaProjects/Bank-AI-Assistant-RAG/rag_env/bin/python3 -m pip install --upgrade pip' command.


Imports

In [2]:

import faiss
import numpy as np
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import chromadb
from chromadb.utils import embedding_functions


/Volumes/nvme1TB_SSD/IdeaProjects/Bank-AI-Assistant-RAG/rag_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Load PDF

In [3]:
def load_pdf(path):
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text

pdf_path = "../../dataset/Bank_of_America_Online Banking_Service Agreement.pdf"  # update path
document_text = load_pdf(pdf_path)
print("Document length (chars):", len(document_text))


Document length (chars): 124022


Chunk Text

In [4]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " "]
)

chunks = splitter.split_text(document_text)
print(f"Total chunks created: {len(chunks)}")
print(chunks[0][:500])  # preview first chunk


Total chunks created: 173
Online Banking
Online Banking Service Agreement
‹‹  Go to Online Banking Overview
Bank of America Online Banking Service Agreement
Effective Date: July 21, 2025
Table of Contents: Hide all Topics
1. General Description of Bank of America Online Banking Service Agreement (this "Agreement")
Introduction
A. What This Agreement Covers
B. Accepting the Agreement
C. Relation to Other Agreements
2. Payment & Transfer Services Using Internal Accounts and Payments to Your Bank of America Loan Accounts Fr


Embeddings (Hugging Face SentenceTransformer)

In [ ]:
!pip install ipywidgets


In [5]:
# Start a local Chroma client (will save to .chromadb folder by default)

chroma_client = chromadb.Client()

# Use Hugging Face MiniLM embeddings
hf_embed = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

# Create or get a collection
collection = chroma_client.get_or_create_collection(
    name="boa_rag",
    embedding_function=hf_embed
)

# Add chunks to the collection
for i, chunk in enumerate(chunks):
    collection.add(documents=[chunk], ids=[str(i)])

print("Collection size:", collection.count())


/Volumes/nvme1TB_SSD/IdeaProjects/Bank-AI-Assistant-RAG/rag_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Collection size: 173


In [ ]:
# You can swap model for bigger one if you have GPU (e.g., mistral-7B)
qa_pipeline = pipeline("text-generation", model="google/flan-t5-base")

def answer_with_context(question, context):
    prompt = f"""
    You are a helpful assistant. Answer the question using ONLY the context below.
    If unsure, say "I don’t know."

    Context:
    {context}

    Question: {question}
    Answer:
    """
    outputs = qa_pipeline(prompt, max_new_tokens=300, do_sample=False)
    return outputs[0]["generated_text"].split("Answer:")[-1].strip()


Define RAG Query Function

In [ ]:
def rag_query(question, top_k=3):
    results = collection.query(query_texts=[question], n_results=top_k)
    retrieved = results["documents"][0]
    context = "\n\n".join(retrieved)
    return answer_with_context(question, context)


Sample Queries

In [ ]:
print(rag_query("What is the Zelle transfer limit for new users?"))
print(rag_query("How do I cancel a scheduled bill payment?"))
print(rag_query("What is the cut-off time for domestic wire transfers?"))
